In [2]:
# Given annual interest rate
annual_interest_rate = 0.054  # 5.4%

# Calculate the monthly interest rate
monthly_interest_rate = (1 + annual_interest_rate)**(1/12) - 1

# Given up and down factors for different time periods
ud_pairs = [
    (1.02, 0.98),
    (1.006, 0.985),
    (1.025, 0.975),
    (1.013, 0.987)
]

# Check the no-arbitrage condition for each pair
no_arbitrage_conditions = [(d < 1 + monthly_interest_rate < u) for u, d in ud_pairs]

monthly_interest_rate, no_arbitrage_conditions


(0.0043923222705009035, [True, True, True, True])

In [3]:
# Calculate q-probabilities for each pair of (u, d)
q_probabilities = [(1 + monthly_interest_rate - d) / (u - d) for u, d in ud_pairs]
q_probabilities = [(q_u, 1 - q_u) for q_u in q_probabilities]

q_probabilities


[(0.6098080567625225, 0.3901919432374775),
 (0.9234439176429, 0.07655608235709999),
 (0.5878464454100193, 0.4121535545899807),
 (0.6689354719423452, 0.33106452805765485)]

In [7]:
from collections import deque

# Given initial stock price and strike price
S0 = 5.35
K = 5.65
n_periods = 20

# Initialize the binary tree with the root being the initial stock price
binomial_tree = deque([(S0, 1)])  # (stock price, period)

# Store the final stock prices to calculate the call option payoff
final_stock_prices = []

# Helper function to calculate call option payoff
def call_option_payoff(S, K):
    return max(S - K, 0)

# Populate the binomial tree
for period in range(1, n_periods + 1):
    # Determine which (u, d) and (q_u, q_d) to use based on the period
    if period % 4 == 1:
        u, d = ud_pairs[0]
        q_u, q_d = q_probabilities[0]
    elif period % 4 == 2:
        u, d = ud_pairs[1]
        q_u, q_d = q_probabilities[1]
    elif period % 4 == 3:
        u, d = ud_pairs[2]
        q_u, q_d = q_probabilities[2]
    else:
        u, d = ud_pairs[3]
        q_u, q_d = q_probabilities[3]
    
    # Number of nodes at the current level
    num_nodes = len(binomial_tree)
    print(num_nodes)
    # Iterate through each node to generate the next level
    for _ in range(num_nodes):
        S, curr_period = binomial_tree.popleft()
        
        # Generate next level stock prices
        Su = S * u
        Sd = S * d
        
        # Append to the queue for next iteration
        binomial_tree.append((Su, curr_period + 1))
        binomial_tree.append((Sd, curr_period + 1))
        
        # If we reach the final period, store the final stock prices for option payoff calculation
        if curr_period == n_periods:
            final_stock_prices.append(S)

# Calculate the call option payoff for each final stock price
final_option_payoffs = [call_option_payoff(S, K) for S in final_stock_prices]

# Show some of the final stock prices and option payoffs for verification
final_stock_prices[:5], final_option_payoffs[:5]


1
2
4
8
16
32
64
128
256
512
1024
2048
4096
8192
16384
32768
65536
131072
262144
524288


([7.2510625720866315,
  6.897352202716552,
  7.099698442848242,
  6.7533716895385725,
  6.966707177102842],
 [1.6010625720866312,
  1.2473522027165513,
  1.4496984428482413,
  1.1033716895385721,
  1.3167071771028418])

In [13]:
# # Reinitialize the parameters and helper functions to rerun the Python code for rechecking

# # Given parameters
# annual_interest_rate = 0.054  # Annual interest rate of 5.4%
# S0 = 5.35  # Initial stock price
# K = 5.65  # Strike price of the European call option
# n_periods = 20  # Number of periods in the binomial tree

# # Calculate the monthly interest rate
# monthly_interest_rate = (1 + annual_interest_rate)**(1/12) - 1

# # Up and down factors (u, d) for different time periods
# ud_pairs = [
#     [1.02, 0.98],
#     [1.006, 0.985],
#     [1.025, 0.975],
#     [1.013, 0.987]
# ]

# # Calculate q-probabilities for each (u, d) pair
# q_probabilities = []
# for u, d in ud_pairs:
#     q_u = (1 + monthly_interest_rate - d) / (u - d)
#     q_d = 1 - q_u
#     q_probabilities.append([q_u, q_d])

# # Helper function to calculate the payoff of a European call option
# def call_option_payoff(S, K):
#     return max(S - K, 0)

# # Initialize a dictionary to store the option price at each node in the binomial tree
# option_prices_py = {}

# # Recursive function to calculate the option price at a given node
# def calculate_option_price_py(S, period):
#     if S in option_prices_py:
#         return option_prices_py[S]
    
#     if period > n_periods:
#         return max(S - K, 0)  # Option payoff at maturity
#     else:
#         # Determine which (u, d) and (q_u, q_d) to use based on the period
#         idx = (period - 1) % 4
#         u, d = ud_pairs[idx]
#         q_u, q_d = q_probabilities[idx]

#         # Calculate the stock prices at the up and down nodes
#         Su = S * u
#         Sd = S * d

#         # Recursive calculation of the option prices at the up and down nodes
#         Cu = calculate_option_price_py(Su, period + 1)
#         Cd = calculate_option_price_py(Sd, period + 1)

#         # Calculate the option price at the current node using risk-neutral pricing
#         price = ((q_u * Cu) + (q_d * Cd)) / (1 + monthly_interest_rate)
        
#         # Cache the option price for future use
#         option_prices_py[S] = price
        
#         return price

# # Calculate the option price at the root node (time 0)
# V0_recheck = calculate_option_price_py(S0, 1)
# V0_recheck


0.2640960660966482

# new

In [4]:
from math import comb
ud_pairs = [
    [1.02, 0.98],
    [1.006, 0.985],
    [1.025, 0.975],
    [1.013, 0.987]
]

def call_option_payoff(S, K):
    return max(S - K, 0)
# # Given parameters
annual_interest_rate = 0.054  # Annual interest rate of 5.4%
S0 = 5.35  # Initial stock price
K = 5.65  # Strike price of the European call option
n_periods = 20  # Number of periods in the binomial tree
# Initialize a list to store the stock prices at each terminal node
terminal_stock_prices = []

monthly_interest_rate = (1 + annual_interest_rate)**(1/12) - 1
q_probabilities = [(1 + monthly_interest_rate - d) / (u - d) for u, d in ud_pairs]
q_probabilities = [(q_u, 1 - q_u) for q_u in q_probabilities]

# Helper function to generate the terminal stock prices in a binomial tree
def generate_terminal_stock_prices(S, period):
    if period == n_periods:
        terminal_stock_prices.append(S)
        return
    else:
        # Determine which (u, d) to use based on the period
        index = (period - 1) % 4
        u, d = ud_pairs[index]
        
        # Generate the stock prices for the up and down movements
        generate_terminal_stock_prices(S * u, period + 1)
        generate_terminal_stock_prices(S * d, period + 1)

# Generate the terminal stock prices
generate_terminal_stock_prices(S0, 1)

# Calculate the terminal option prices and their probabilities
terminal_option_prices = [call_option_payoff(S, K) for S in terminal_stock_prices]
print(len(terminal_stock_prices))
# Initialize the option price at time 0
V0_fixed = 0

# Calculate the option price at time 0 using the terminal option prices and probabilities
for j in range(n_periods + 1):
    # Calculate the number of combinations (n choose j)
    num_combinations = comb(n_periods, j)
    
    # Determine which (q_u, q_d) to use based on j
    index = j % 4
    q_u, q_d = q_probabilities[index]
    
    # Calculate the probability of this specific path
    path_probability = (q_u ** j) * (q_d ** (n_periods - j))
    
    # Add to the option price at time 0
    V0_fixed += num_combinations * path_probability * terminal_option_prices[j]

# Discount the option price back to time 0
V0_fixed = V0_fixed / ((1 + monthly_interest_rate) ** n_periods)
V0_fixed


524288


0.8216198105440476

In [8]:

V0 = 0
for j in range(n_periods+1):
    u, d = ud_pairs[j%4]
    q_u, q_d = q_probabilities[j%4]
    V0 += call_option_payoff(S0*(u**j)*(d**(n_periods-j)), K)*comb(n_periods, j)*(q_u**j)*(q_d**(n_periods-j))
V0 = V0/((1+monthly_interest_rate)**n_periods)
V0

0.21681470307832845